In [5]:
import pandas as pd
import random

In [6]:
# Importing the dataset
df_pokemon = pd.read_csv('Pokemon_Gen1-6.csv')
df_pokemon.head(n=5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [7]:
# Cheking the data structure
print(df_pokemon.columns)
print(df_pokemon.index)

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')
RangeIndex(start=0, stop=800, step=1)


In [8]:
# Filtering out the non-legendaries
df_non_legendary = df_pokemon[df_pokemon['Legendary'] == False]
df_non_legendary.head(n=5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [9]:
## Generating possible team of 6 (without extra criteria, randomized from 801 Pokemons)

# Define team size
team_size = 6

# Setting seed
random.seed(40)

# Randomize team
random_team = df_non_legendary.sample(n=team_size, random_state=40) #change the random_state to reproduce another output
print(random_team)


       #       Name    Type 1 Type 2  Total  HP  Attack  Defense  Sp. Atk  \
394  360     Wynaut   Psychic    NaN    260  95      23       48       23   
779  709  Trevenant     Ghost  Grass    474  85     110       76       65   
345  316     Gulpin    Poison    NaN    302  70      43       53       43   
341  312      Minun  Electric    NaN    405  60      40       50       75   
343  314   Illumise       Bug    NaN    400  65      47       55       73   
576  517      Munna   Psychic    NaN    292  76      25       45       67   

     Sp. Def  Speed  Generation  Legendary  
394       48     23           3      False  
779       82     56           6      False  
345       53     40           3      False  
341       85     95           3      False  
343       75     85           3      False  
576       55     24           5      False  


In [10]:
## Generating possible team of 6 with added criterias:
# 1) No more than 1 Pokemon has the same Type1
# 2) Each pokemon must excel in one stat (HP, Attack, Def, SpAtk, SpDef, Speed) - That stat must be above average of the other Pokemons

# Computing average stats
stats_column = ['HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed']
stats_average = df_non_legendary[stats_column].mean()
print(stats_average)

HP         67.182313
Attack     75.669388
Defense    71.559184
Sp. Atk    68.454422
Sp. Def    68.892517
Speed      65.455782
dtype: float64


In [11]:
# Finding Pokemons that excel in at least 1 stats
def excels_in_one_stats(row,averages):
  return any(row[stats] > averages[stats] for stats in stats_column)

df_excels = df_non_legendary[df_non_legendary.apply(excels_in_one_stats, axis=1, averages=stats_average)]
print(df_excels)

       #                   Name  Type 1  Type 2  Total  HP  Attack  Defense  \
1      2                Ivysaur   Grass  Poison    405  60      62       63   
2      3               Venusaur   Grass  Poison    525  80      82       83   
3      3  VenusaurMega Venusaur   Grass  Poison    625  80     100      123   
5      5             Charmeleon    Fire     NaN    405  58      64       58   
6      6              Charizard    Fire  Flying    534  78      84       78   
..   ...                    ...     ...     ...    ...  ..     ...      ...   
786  711    GourgeistLarge Size   Ghost   Grass    494  75      95      122   
787  711    GourgeistSuper Size   Ghost   Grass    494  85     100      122   
788  712               Bergmite     Ice     NaN    304  55      69       85   
789  713                Avalugg     Ice     NaN    514  95     117      184   
791  715                Noivern  Flying  Dragon    535  85      70       80   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary

In [12]:
# Selecting unique Type 1 Pokemons
unique_types = df_excels['Type 1'].unique()
print(unique_types)

['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']


In [16]:
# Randomly select one Pokemon from Type 1 until we get a team of 6
random_team = []

for i in range(team_size):
    selected_type = random.choice(unique_types) # Randomize type
    type_pokemon = df_excels[df_excels['Type 1'] == selected_type] # Only takes the filtered Pokemons

    if not type_pokemon.empty:
        chosen_pokemon = type_pokemon.sample(n=1, random_state=42).iloc[0] # Randomly select one Pokémon from the filtered list
        random_team.append(chosen_pokemon)
        unique_types = unique_types[unique_types != selected_type]  # Remove selected type to avoid duplicates

df_random_team = pd.DataFrame(random_team)
print(df_random_team)

       #                 Name    Type 1  Type 2  Total   HP  Attack  Defense  \
223  208              Steelix     Steel  Ground    510   75      85      200   
640  579            Reuniclus   Psychic     NaN    490  110      65       75   
244  226              Mantine     Water  Flying    465   65      40       70   
29    24                Arbok    Poison     NaN    438   60      85       69   
327  302  SableyeMega Sableye      Dark   Ghost    480   50      85      125   
536  479       RotomMow Rotom  Electric   Grass    520   50      65      107   

     Sp. Atk  Sp. Def  Speed  Generation  Legendary  
223       55       65     30           2      False  
640      125       85     30           5      False  
244       80      140     70           2      False  
29        65       79     80           1      False  
327       85      115     20           3      False  
536      105      107     86           4      False  
